In [1]:
import keras
import numpy as np
from sequence_attention import SeqAttModel, preprocess_data, preprocess_data_pickle
from sequence_attention import DataGenerator, DataGeneratorUnlabeled, DataGeneratorPickle, DataGeneratorUnlabeledPickle
from config import Config

opt = Config()

Using TensorFlow backend.


In [2]:
preprocess_data(opt)

# preprocess_data_pickle(opt)

26-Oct-20 23:48:37 - Processing raw data: 0.0% completed.
26-Oct-20 23:59:58 - Processing raw data: 10.0% completed.
27-Oct-20 00:11:09 - Processing raw data: 20.0% completed.
27-Oct-20 00:22:22 - Processing raw data: 30.0% completed.


In [3]:
import pickle
label_dict = pickle.load(open('{}/label_dict.pkl'.format(opt.out_dir), 'rb')) 
sample_to_label, read_meta_data = pickle.load(open('{}/meta_data.pkl'.format(opt.out_dir), 'rb'))
partition = pickle.load(open('{}/train_test_split.pkl'.format(opt.out_dir), 'rb')) 
seq_att_model = SeqAttModel(opt)

27-Oct-20 00:39:23 - Model initialized.


In [6]:
training_generator = DataGenerator(partition['train'], sample_to_label, label_dict, 
                                   dim=(opt.SEQLEN,opt.BASENUM), batch_size=opt.batch_size, shuffle=opt.shuffle)
testing_generator = DataGenerator(partition['test'], sample_to_label, label_dict, 
                                   dim=(opt.SEQLEN,opt.BASENUM), batch_size=opt.batch_size, shuffle=opt.shuffle)


In [4]:
training_generator = DataGeneratorPickle(partition['train'], sample_to_label, label_dict, 
                                   dim=(opt.SEQLEN,opt.BASENUM), batch_size=opt.batch_size, shuffle=opt.shuffle)
testing_generator = DataGeneratorPickle(partition['test'], sample_to_label, label_dict, 
                                   dim=(opt.SEQLEN,opt.BASENUM), batch_size=opt.batch_size, shuffle=opt.shuffle)


In [7]:
X, y = training_generator.__getitem__(0)

In [10]:
y.shape

(1024,)

In [ ]:
def preprocess_data(opt):
    '''
    preprocessing the data
    '''
    
    if not os.path.exists(opt.out_dir):
        os.makedirs(opt.out_dir)
    
    meta_data = pd.read_csv('{}/meta_data.csv'.format(opt.in_dir), dtype='str')
    
    label_list = sorted(meta_data['label'].unique())
    
    label_dict = {}
    for idx, label in enumerate(label_list):
        label_dict[label] = idx
        
    pickle.dump(label_dict, open('{}/label_dict.pkl'.format(opt.out_dir), 'wb'))
    
    for label in label_list:
        label_dir = '{}/{}'.format(opt.out_dir, label)
        if os.path.exists(label_dir):
            continue
        os.makedirs(label_dir)
        
    read_meta_data = {}
    STEP = meta_data.shape[0] // 10 if meta_data.shape[0] > 10 else 1
    for idx in range(meta_data.shape[0]):
        if idx % STEP == 0:
            logging.info('Processing raw data: {:.1f}% completed.'.format(10 * idx / STEP))
        sample_id, label = meta_data.iloc[idx]['sample_id'], meta_data.iloc[idx]['label']

        read_meta_data[sample_id] = fna_to_dict(sample_id, label, opt.in_dir, opt.out_dir)
    
    min_num_sample = min([meta_data[meta_data['label']==label].shape[0] for label in label_list])
    
    num_train_samples_per_cls = opt.num_train_samples_per_cls
    num_train_samples_per_cls = num_train_samples_per_cls if num_train_samples_per_cls < min_num_sample else min_num_sample
    
    partition = train_test_split(meta_data, num_train_samples_per_cls)
    
    
    sample_to_label = {}
    for idx in range(meta_data.shape[0]):
        sample_id, label = meta_data.iloc[idx]['sample_id'], meta_data.iloc[idx]['label']
        sample_to_label[sample_id] = label

    pickle.dump([sample_to_label, read_meta_data], open('{}/meta_data.pkl'.format(opt.out_dir), 'wb'))
    
    train_list = []
    for sample_id in partition['train']:
        train_list.extend([('{}/{}/{}.pkl'.format(opt.out_dir, sample_to_label[sample_id], sample_id), read_id) for read_id in read_meta_data[sample_id]])
    test_list = []
    for sample_id in partition['test']:
        test_list.extend([('{}/{}/{}.pkl'.format(opt.out_dir, sample_to_label[sample_id], sample_id), read_id) for read_id in read_meta_data[sample_id]])
    
    read_partition = {'train': train_list, 'test': test_list}
    pickle.dump(read_partition, open('{}/train_test_split.pkl'.format(opt.out_dir), 'wb'))   

In [ ]:
def fna_to_dict(sample_id, label, in_dir, out_dir):
    '''
    convert fna file to dictionary
    '''
    
    out_name = '{}/{}/{}.pkl'.format(out_dir, label, sample_id)
    
    meta_data_read_list = []
    read = ''
    f_sample = open(filename)
    read_dict = {}
    for line in f_sample:
        if line[0] == '>':
            if len(read) != 0:
                read_dict[header] = read
                meta_data_read_list.append(header)
                read = ''
            read += line
            header = line[1:].strip()
        else:
            read += line
    if len(read) != 0:
        read_dict[header] = read
        meta_data_read_list.append(header)
    f_sample.close()
    pickle.dump(read_dict, open(out_name, 'wb'))
    return meta_data_read_list

In [ ]:
X = np.zeros((self.batch_size, *self.dim))
if self.n_classes == 2:
    y = np.zeros((self.batch_size,), dtype=int)
else:
    y = np.zeros((self.batch_size, self.n_classes), dtype=int)
# Generate data
for i, file in enumerate(list_IDs_temp):
    # Store sample
    sample_file_name = file[0]
    sample_id = sample_file_name.split('/')[-1][:-4]
    header = file[1]
    read_dict = pickle.load(open(sample_file_name, 'rb'))
    seq = read_dict[header]

    seq_tmp = seq if len(seq) < self.dim[0] else seq[:self.dim[0]]
    seq_coded = self.__seq_mapper(seq_tmp)
    X[i,] = seq_coded
    if self.n_classes == 2:
        y[i] = self.sample_to_label[sample_id]
    else:
        y[i,self.label_dict[self.sample_to_label[sample_id]]] = 1

return X, y 